In [ ]:
# ------------------------------------------------------------
# exogenous_train.ipynb - 환경/학습 파라미터/데이터 로드 셋업 셀
# 목적:
#   - 프로젝트 경로/저장 경로 설정
#   - 학습에 사용할 기본 하이퍼파라미터(lookback, horizon, batch_size 등) 정의
#   - (선택) 과거/미래 외생변수 설정
#   - 학습에 사용할 실데이터 parquet 로드
#
# 작업자가 이 셀에서 해야 할 일:
#   1) dir / save_root 를 본인 PC 경로에 맞게 수정
#   2) plan_week(기준 주차)와 lookback/horizon이 실험 의도와 맞는지 확인
#   3) df 파일 경로 및 컬럼(id/date/y)이 실제 parquet 스키마와 일치하는지 확인
#   4) 외생변수 전략:
#       - past_exo_cont_cols / past_exo_cat_cols 를 실제 사용 컬럼으로 채우거나
#       - 미래 외생은 future_exo_cb(캘린더 기반)로 생성하는지 결정
# ------------------------------------------------------------

from modeling_module.utils.exogenous_utils import compose_exo_calendar_cb
# - 미래 외생변수(future exogenous) 생성용 콜백을 만들어주는 유틸
# - ex) date_type='W', sincos=True이면 주차 기반 sin/cos 캘린더 피처를 horizon 길이만큼 생성

import os
import time
from datetime import date

import numpy as np
import polars as pl
import torch
import sys


# ------------------------------------------------------------
# (1) 프로젝트 기본 경로 및 저장 경로 설정
# ------------------------------------------------------------
dir = "C:/Users/USER/PycharmProjects/ts_forecaster_lib/raw_data/"  # 프로젝트 raw_data 루트 경로
# - 작업자 변경 포인트:
#   - 본인 PC/서버 환경에 맞게 경로를 반드시 수정
#   - Windows/리눅스 경로 구분 주의 (예: 리눅스는 /home/... 형태)

save_dir = os.path.join(dir, 'fit')
# - 학습 결과(모델 ckpt, 로그 등)를 저장할 상위 디렉토리

os.makedirs(save_dir, exist_ok=True)
# - save_dir가 없으면 생성 (재실행해도 에러 나지 않도록 exist_ok=True)

save_root = os.path.join(save_dir, 'Xpatchtst', '20260119')
# - 실험 단위로 구분되는 실제 저장 루트
# - 작업자 변경 포인트:
#   - 'Xpatchtst'는 실험 카테고리/모델명 prefix로 사용 가능
#   - '20260119'는 실험 날짜/버전 태그로 사용 (재현성/추적을 위해 권장)


# ------------------------------------------------------------
# (2) 학습/데이터 기준 시점 및 공통 하이퍼파라미터
# ------------------------------------------------------------
plan_week = 201103
# - 기준 주차(YYYYWW 형태)로 해석되는 값
# - 보통 "예측 시작 주차" 또는 "validation cut-off" 등을 의미할 수 있음
# - 작업자 확인 포인트:
#   - 이 값이 실제 df의 date 범위와 호환되는지(존재하는 주차인지) 확인
#   - 코드 어딘가에서 plan_week를 실제 split/샘플링 기준으로 사용한다면 매우 중요

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# - GPU 사용 가능하면 cuda, 아니면 cpu로 학습
# - 작업자 확인 포인트:
#   - 서버에서 CUDA 인식이 안 되면 torch.cuda.is_available()가 False로 떨어짐
#   - 예상과 다르면 드라이버/CUDA/PyTorch 설치 확인 필요

lookback = 52
# - 과거 입력 길이 (weekly면 52주 히스토리 사용)

horizon = 27
# - 미래 예측 길이 (weekly면 27주 ahead 예측)

batch_size = 256
# - 배치 크기
# - 작업자 조정 포인트:
#   - GPU 메모리 부족(OOM) 시 128/64로 감소
#   - 안정적 학습이 필요하면 너무 큰 배치는 오히려 학습이 둔해질 수 있음(실험적으로 조정)

freq = 'weekly'
# - 데이터 주기
# - DataModule / date util이 freq에 따라 date 처리 로직이 달라질 수 있음

split_mode = 'multi'
# - 데이터 split 전략 (프로젝트 구현체에 따라 의미가 정의됨)
# - 일반적으로:
#   - 'multi': 여러 시계열을 함께 학습/평가하는 모드
#   - 'single': 특정 1개 시계열/파트만 대상으로 하는 모드
# - 작업자 확인 포인트:
#   - 프로젝트에서 split_mode별 train/val 분할 방식이 다르면 결과가 크게 달라짐

shuffle = True
# - DataLoader shuffle 여부
# - 일반적으로 train은 True, val은 False가 권장
# - 단, 시계열 윈도우 샘플링 방식에 따라 shuffle이 실험 재현성에 영향을 줄 수 있음


# ------------------------------------------------------------
# (3) 데이터 컬럼명 정의 (df 스키마와 반드시 일치해야 함)
# ------------------------------------------------------------
id_col = 'unique_id'
# - 각 시계열을 구분하는 ID 컬럼명 (예: 상품/점포/파트 번호 등)

date_col = 'date'
# - 시간 축 컬럼명
# - weekly면 YYYYWW(int) 혹은 date 타입이 올 수 있음
# - 프로젝트 DataModule이 요구하는 타입/포맷(정수 YYYYWW vs date)을 사전에 확인 필요

y_col = 'y'
# - 타깃(수요/매출) 컬럼명


# ------------------------------------------------------------
# (4) 외생변수(Exogenous) 설정
# ------------------------------------------------------------

# add past exogenous continuous variable columns
past_exo_cont_cols = ()
# - 과거 구간(lookback)에 대해 함께 입력으로 넣을 "연속형" 외생변수 컬럼명 리스트/튜플
# - 예: ('price', 'promo_rate', 'stock', ...)
# - 현재는 비어있으므로 과거 외생은 사용하지 않음
# - 작업자 변경 포인트:
#   - df에 해당 컬럼이 존재할 때만 추가
#   - 스케일/정규화 여부(특히 가격/재고 등)도 함께 고려

# add past exogenous categorical variable columns
past_exo_cat_cols = ()
# - 과거 구간에 대해 함께 입력으로 넣을 "범주형" 외생변수 컬럼명 리스트/튜플
# - 예: ('store_type', 'region', ...)
# - 일반적으로 embedding으로 들어가며, cardinality 관리가 필요할 수 있음

future_exo_cb = compose_exo_calendar_cb(date_type='W', sincos=True)
# - 미래 구간(horizon)에 대해 date로부터 외생을 생성하는 callable 콜백
# - date_type='W': 주차 기반 캘린더 피처 생성
# - sincos=True : 주기성 정보를 sin/cos로 인코딩 (값 범위 대체로 [-1, 1])
# - 작업자 확인 포인트:
#   - DataLoader/Collate가 실제로 이 콜백을 사용하도록 연결되어 있는지(= future exo 활성화)
#   - 모델이 미래 외생을 받을 구조(exo_head 등)를 제대로 빌드하는지
#   - fe_dim이 0으로 나오는 경우: 콜백이 비활성/무시되고 있을 수 있음


# ------------------------------------------------------------
# (5) 실데이터 로드
# ------------------------------------------------------------
df = pl.read_parquet()
# - 학습에 사용할 parquet 데이터 로드
# - 작업자 확인 포인트(필수):
#   1) 파일 경로가 실제 존재하는지
#   2) df 컬럼에 id_col/date_col/y_col이 존재하는지
#   3) weekly 데이터면 date_col이 YYYYWW로 잘 구성되어 있는지(누락/형변환 이슈)
#   4) 외생 컬럼을 추가할 계획이면, 해당 컬럼들이 df에 포함되어 있는지
#
# 권장(추가 점검 예시):
#   - print(df.shape, df.columns)
#   - df.select([id_col, date_col, y_col]).head()


In [ ]:
from modeling_module.training.forecater import DMSForecaster
import numpy as np
import polars as pl
import torch
from datetime import date, timedelta

# ============================================================
# [셀 목적]
# 1) YYYYWW(ISO Week) 형태의 주차 정수 ↔ 날짜(date) 변환 유틸 제공
# 2) inference_loader에서 배치를 받아
#    - base_model(포인트 예측)
#    - quantile_model(q50 예측)
#    을 각각 추론한 뒤
#    (oper_part_no, forecast_week) 단위의 "long-format 결과 테이블"을 생성
#
# [작업자가 이 셀에서 해야 할 일]
# A. inference_loader 배치 구조가 아래 가정과 동일한지 확인
#    - 현재 가정: (x, uid, fe, pe_cont, pe_cat)
#      * 과거 코드에서 train loader는 (x, y, uid, fe, pe_cont, pe_cat)였음
#      * inference용 loader는 y가 없을 수 있으니 이 형태로 가정
#    - 만약 loader가 uid를 tensor가 아닌 string list로 준다면 pid 처리 로직 수정 필요
#
# B. plan_week의 의미/정의를 명확히 유지
#    - "예측 기준 주차"로 보고, forecast_week = plan_week + h (h=0..H-1)로 생성
#    - 만약 현업에서 plan_week + 1부터 예측해야 하면 range 시작을 1로 바꿔야 함
#
# C. 모델 입력에 외생변수(fe/pe_cont/pe_cat)를 실제로 넣을지 결정
#    - loader에서 fe_dim=0이면 future_exo_batch가 빈 텐서일 수 있음
#    - 모델이 exo를 기대하는데 fe가 비어있으면 성능/동작에 영향
#
# D. 성능/속도 이슈 점검
#    - 현재 구현은 배치 내 각 샘플을 1개씩(1-step) predict 호출 -> 느릴 수 있음
#    - 대량 inference는 "배치 단위로 predict"로 리팩토링 권장
# ============================================================


# -----------------------------
# YYYYWW(ISO week) 유틸
# -----------------------------
def yyyyww_to_monday(yyyyww: int) -> date:
    """
    YYYYWW(정수) -> 해당 ISO week의 월요일(date)로 변환.

    예) 202601 -> 2026년 ISO week 1의 월요일 날짜

    [주의]
    - ISO week는 연말/연초에 '연도'가 달라질 수 있음(ISO year 기준)
      예: 2025년 12월 말이 ISO 기준으로 2026년 1주에 포함될 수 있음
    - 입력 주차가 존재하지 않는 값이면 date.fromisocalendar에서 ValueError 발생
    """
    y = int(yyyyww) // 100
    w = int(yyyyww) % 100
    return date.fromisocalendar(y, w, 1)  # ISO week에서 월요일=1


def monday_to_yyyyww(d: date) -> int:
    """
    date -> YYYYWW(정수)로 변환(ISO year-week 기준)

    예) 2026-01-05(월) -> 202601
    """
    iso_y, iso_w, _ = d.isocalendar()
    return int(iso_y) * 100 + int(iso_w)


def add_week(yyyyww: int, add: int) -> int:
    """
    YYYYWW에 주(week) 단위로 add만큼 더한 YYYYWW 반환.

    예) add_week(202601, 1) -> 202602

    [작업자 변경 가능 포인트]
    - 예측 결과를 plan_week 포함(H개) vs plan_week+1부터(H개)로 만들지 정책에 따라 달라짐
    """
    return monday_to_yyyyww(yyyyww_to_monday(yyyyww) + timedelta(weeks=int(add)))


# -----------------------------
# 결과 테이블 생성 함수
# -----------------------------
@torch.no_grad()
def make_forecast_result_table(
    *,
    inference_loader,
    base_model: torch.nn.Module,
    quantile_model: torch.nn.Module,
    plan_week: int,
    horizon: int,
    device: str = "cuda",
    max_parts: int = 10_000,          # 필요시 제한
) -> pl.DataFrame:
    """
    [기능 요약]
    inference_loader에서 배치를 순회하며 각 시계열(파트/스토어 등)별로
    - base_model: point forecast
    - quantile_model: q50 forecast
    을 생성하고,
    (plan_week, oper_part_no, forecast_week) 단위 long table로 반환한다.

    [입력 가정]
    inference_loader는 batch마다 아래 형태를 반환한다고 가정:
        (x, uid, fe, pe_cont, pe_cat)
    - x: (B, L, C)  과거 타깃(및 채널)
    - uid: (B,)     각 샘플 식별자
    - fe: (B, H, E) 미래 외생(future exo). 없으면 None 또는 shape[-1]=0 가능
    - pe_cont: (B, L, E_past_cont) 과거 연속 외생
    - pe_cat: (B, L, E_past_cat)   과거 범주 외생

    [출력]
    polars DataFrame, long format:
        plan_week: int
        oper_part_no: str
        forecast_week: int
        base_forecast: float
        quantile_forecast: float

    [작업자 주의사항]
    1) plan_week 포함 여부:
       - 현재 weeks = [plan_week + 0, ..., plan_week + H-1]
       - 만약 "예측은 다음 주부터"라면 range를 1..H 로 바꿔야 함

    2) uid 타입:
       - uid가 torch.Tensor scalar면 pid.item() 사용 가능
       - uid가 이미 문자열이면 그대로 str(uid) 처리해야 함

    3) 성능:
       - 배치 내 샘플을 i 루프로 1개씩 예측 -> 느림
       - production/대량 inference면 DMSForecaster가 batch 입력을 받도록 개선 권장
    """

    # --------------------------------------------------------
    # (1) Forecaster 래핑
    # --------------------------------------------------------
    # DMSForecaster는 모델 추론을 위한 공통 wrapper로 보이며,
    # - target_channel=0: y 채널이 다변량일 때 어떤 채널을 예측 대상으로 볼지 지정
    # - fill_mode="copy_last": 입력 부족/결측 시 마지막 값 복사 등 보정 전략 (프로젝트 정책 확인)
    base_fc = DMSForecaster(base_model, target_channel=0, fill_mode="copy_last")
    q_fc    = DMSForecaster(quantile_model, target_channel=0, fill_mode="copy_last")

    rows = []
    n_parts = 0

    # --------------------------------------------------------
    # (2) inference_loader 순회
    # --------------------------------------------------------
    for batch in inference_loader:
        # [중요] 여기서 배치 언패킹 형태가 loader 구현과 다르면 바로 에러 납니다.
        # - 만약 loader가 (x, y, uid, fe, pe_cont, pe_cat) 형태면 아래 줄을 수정해야 합니다.
        x, uid, fe, pe_cont, pe_cat = batch

        # B: 배치 크기
        B = x.size(0)

        # ----------------------------------------------------
        # (3) 배치 내 샘플(=파트/시계열) 단위 순회
        # ----------------------------------------------------
        for i in range(B):
            # max_parts를 넘기면 early stop
            if n_parts >= max_parts:
                break

            # (3-1) 샘플 1개만 분리하여 (1, L, C) 형태로 만듦
            # - DMSForecaster.predict가 batch=1 입력을 받아 처리하도록 구성된 것으로 가정
            x1 = x[i:i+1]

            # (3-2) 파트/시계열 ID 추출
            # - uid가 tensor scalar라면 uid[i]는 tensor(0-d) 또는 tensor(1,)일 수 있음
            # - uid가 str/list면 uid[i] 자체가 string일 수 있음
            pid = uid[i]  # oper_part_no로 사용(필요하면 mapping)

            # (3-3) 외생변수도 같은 인덱스로 1개 샘플만 분리
            # - fe/pe_cont/pe_cat가 None이면 그대로 None
            fe1  = fe[i:i+1] if fe is not None else None
            pec1 = pe_cont[i:i+1] if pe_cont is not None else None
            pek1 = pe_cat[i:i+1] if pe_cat is not None else None

            # ------------------------------------------------
            # (4) Base(Point) 예측
            # ------------------------------------------------
            pred_base = base_fc.predict(
                x1,
                horizon=int(horizon),
                device=device,
                mode="eval",
                # part_ids는 forecaster 내부에서 로깅/조건 분기 등에 사용될 수 있음
                # - uid 타입이 tensor면 list에 tensor가 들어갈 수 있어 내부에서 str 변환 필요할 수 있음
                part_ids=[pid] if pid is not None else None,
                # 과거 외생
                past_exo_cont=pec1,
                past_exo_cat=pek1,
                # 미래 외생 (H, E)
                future_exo_batch=fe1,
            )

            # pred_base["point"]를 (H,)로 변환
            # - 반환 shape이 (1, H) or (H,) 등 다양할 수 있으므로 reshape(-1)로 평탄화
            y_base = np.asarray(pred_base["point"]).reshape(-1)

            # ------------------------------------------------
            # (5) Quantile(q50) 예측
            # ------------------------------------------------
            pred_q = q_fc.predict(
                x1,
                horizon=int(horizon),
                device=device,
                mode="eval",
                part_ids=[pid] if pid is not None else None,
                past_exo_cont=pec1,
                past_exo_cat=pek1,
                future_exo_batch=fe1,
            )

            # - quantile 모델은 q50 키가 있을 수도 있고 없을 수도 있음(프로젝트 구현체에 따라 다름)
            # - q50이 없으면 point를 fallback으로 사용
            y_q50 = np.asarray(pred_q.get("q50", pred_q["point"])).reshape(-1)

            # ------------------------------------------------
            # (6) forecast_week 생성
            # ------------------------------------------------
            # 현재는 plan_week 포함하여 horizon개 생성:
            #   [plan_week, plan_week+1, ..., plan_week+(H-1)]
            # 정책상 "다음주부터" 예측이면:
            #   weeks = [add_week(plan_week, h) for h in range(1, H+1)]
            weeks = [add_week(plan_week, h) for h in range(int(horizon))]

            # ------------------------------------------------
            # (7) long-format row 적재
            # ------------------------------------------------
            # uid가 tensor(1개 원소)라면 pid.item()으로 파이썬 스칼라 변환 후 문자열화
            # uid가 이미 문자열이면 str(pid)로 충분
            pid_str = str(pid.item()) if torch.is_tensor(pid) and pid.numel() == 1 else str(pid)

            # weeks, y_base, y_q50 길이가 모두 horizon인지 확인 필요
            # - 불일치 시 zip이 짧은 쪽에 맞춰 잘리므로 조용히 데이터가 누락될 수 있음
            for w, bval, qval in zip(weeks, y_base.tolist(), y_q50.tolist()):
                rows.append({
                    "plan_week": int(plan_week),
                    "oper_part_no": pid_str,
                    "forecast_week": int(w),
                    "base_forecast": float(bval),
                    "quantile_forecast": float(qval),
                })

            n_parts += 1

        if n_parts >= max_parts:
            break

    # --------------------------------------------------------
    # (8) polars DataFrame 반환
    # --------------------------------------------------------
    # rows가 비어있으면 빈 DF가 생성됨
    return pl.DataFrame(rows)


In [ ]:
from modeling_module.utils.checkpoint import load_model_dict
from modeling_module.models import build_patchTST_quantile, build_patchTST_base
from modeling_module.data_loader import MultiPartExoDataModule
from modeling_module.training.model_trainers.total_train import run_total_train_weekly

# ============================================================
# [셀 목적]
# 1) "특정 plan_week 시점" 기준으로 inference_loader를 생성한다.
# 2) 저장된 체크포인트(.pt 등)를 로드하여
#    - base_model (PatchTST point forecast)
#    - quantile_model (PatchTST quantile forecast: q10/q50/q90 등)
#    을 준비한다.
# 3) make_forecast_result_table()을 호출해
#    long-format 결과 테이블(pl.DataFrame)을 만든다.
#
# [작업자가 이 셀에서 확인/수정해야 할 핵심 포인트]
# A. inference_loader 배치 구조 확인
#    - 여기서는 (x, uid, fe, pe_cont, pe_cat)로 가정하고 있음
#    - 만약 (x, y, uid, fe, pe_cont, pe_cat)라면 inspect()와 아래 로직 수정 필요
#
# B. plan_week(plan_dt)의 의미/정합성
#    - get_inference_loader_at_plan(plan_dt=plan_week) 호출에서 plan_dt가 "YYYYWW"임을 전제로 함
#    - 내부 구현이 YYYYMMDD 또는 다른 정수 포맷이면 compose_exo_calendar_cb(date_type='W')와 충돌 가능
#
# C. save_root 경로 및 builders key 일치 여부
#    - load_model_dict()가 디렉토리 구조/파일명 규칙에 의존할 가능성이 큼
#    - builders의 key('patchtst', 'patchtst_quantile')가 체크포인트 메타/파일명과 매칭되는지 확인
#
# D. device 일관성
#    - model_loader에서 device로 로드했더라도, make_forecast_result_table에서 device를 동일하게 전달해야 함
#
# E. 외생변수(exogenous) 사용 여부
#    - data_module에 future_exo_cb를 넘기고 있으므로 fe가 생성될 수 있음
#    - 모델이 exo 입력을 기대하는 설정인지(model config)와 일치해야 함
# ============================================================


# ------------------------------------------------------------
# (0) 체크포인트 로더/빌더 준비
# ------------------------------------------------------------
# load_model_dict:
# - save_root 아래에 저장된 여러 모델 체크포인트를 한 번에 로드하는 유틸로 추정
# - builders: {모델이름: build_fn} 형태로 "아키텍처 생성 함수"를 넘겨주면,
#            체크포인트의 state_dict를 해당 아키텍처에 로드하여 반환하는 형태일 가능성이 높음
#
# build_patchTST_base:
# - point forecast(단일 값)용 PatchTST 생성
# build_patchTST_quantile:
# - quantile forecast(q10/q50/q90 등)용 PatchTST 생성
#
from modeling_module.utils.checkpoint import load_model_dict
from modeling_module.models import build_patchTST_quantile, build_patchTST_base


# ------------------------------------------------------------
# (1) inference_loader 구성: MultiPartExoDataModule
# ------------------------------------------------------------
# MultiPartExoDataModule:
# - df를 (id_col, date_col) 기준으로 시계열로 묶어 샘플링
# - lookback 길이의 과거 입력(x)과 horizon 길이의 미래 구간(y or placeholder)을 구성
# - future_exo_cb가 있으면 horizon 구간의 미래 외생(fe)을 생성/결합
#
# [작업자 체크]
# - df, id_col/date_col/y_col, lookback/horizon 등은 "이 셀 이전"에서 정의되어 있어야 함
# - past_exo_cont_cols/past_exo_cat_cols가 빈 tuple이면 pe_cont/pe_cat이 0차원일 수 있음
#
from modeling_module.data_loader import MultiPartExoDataModule

def inspect(loader, name):
    """
    loader가 실제로 어떤 텐서를 뱉는지 빠르게 확인하는 디버그 함수.

    [가정]
    - inference_loader는 (x, uid, fe, pe_cont, pe_cat) 형태를 반환한다고 가정
      * 학습 loader는 보통 y가 포함되지만, inference에서는 y 없이 구성하는 경우가 많음
    - x: (B, L, C)
    - uid: (B,) 또는 list[str]
    - fe: (B, H, E)
    - pe_cont: (B, L, E_past_cont)
    - pe_cat: (B, L, E_past_cat)

    [작업자 참고]
    - 만약 여기서 unpacking 에러가 나면 loader 배치 구조가 다른 것임
      -> batch print 후 구조에 맞게 수정 필요
    """
    b = next(iter(loader))

    # inference_loader 배치 구조 언패킹
    x, uid, fe, pe_cont, pe_cat = b

    print(f"[{name}] x:", x.shape, x.device, x.dtype)
    print(f"[{name}] fe:", fe.shape, fe.device, fe.dtype)
    print(f"[{name}] pe:", pe_cont.shape, pe_cont.device, pe_cont.dtype)

    # train_loader에서는 loader.collate_fn.future_exo_cb를 체크했으나
    # inference_loader 구현에 따라 collate_fn 속성이 다를 수 있어 주석 처리
    # print(f"[{name}] future_exo_cb is None?", loader.collate_fn.future_exo_cb is None)

    # fe_dim > 0이면 실제 값 일부를 출력해 "미래 외생이 채워졌는지" 확인
    # - sin/cos 달력 외생이라면 [-1, 1] 범위의 값이 나오는 것이 일반적
    if fe.shape[-1] > 0:
        print(f"[{name}] fe sample:", fe[0, :3, :])


data_module = MultiPartExoDataModule(
    df=df,
    id_col=id_col,
    date_col=date_col,
    y_col=y_col,
    lookback=lookback,
    horizon=horizon,
    batch_size=batch_size,
    past_exo_cont_cols=past_exo_cont_cols,
    past_exo_cat_cols=past_exo_cat_cols,
    future_exo_cb=future_exo_cb,   # 달력 기반 미래 외생 생성 콜백(weekly + sin/cos)
    freq=freq,                     # 'weekly' 등. future_exo_cb의 date_type과 정합성 필수
    shuffle=shuffle,
    split_mode=split_mode,         # 'multi' 등. 데이터 분할/샘플링 전략
)

# ------------------------------------------------------------
# (2) 특정 plan_week 기준 inference_loader 생성
# ------------------------------------------------------------
# get_inference_loader_at_plan(plan_dt=plan_week):
# - "plan_week 시점"에 대해 예측에 필요한 x/fe/pe_*를 구성해서 loader 반환
#
# [작업자 체크]
# - plan_week가 YYYYWW로 들어가고, 내부도 이를 기대해야 함
# - plan_week 시점에 해당하는 lookback 히스토리가 충분히 존재하는지 확인 필요
#
inference_loader = data_module.get_inference_loader_at_plan(plan_dt=plan_week)

# 필요 시 배치 구조를 직접 확인(문제 발생 시 주석 해제)
# for batch in inference_loader:
#     print(batch)
#     break

# 배치 shape 확인
inspect(inference_loader, 'inference_loader')


# ------------------------------------------------------------
# (3) 모델 빌더 매핑 구성
# ------------------------------------------------------------
# builders:
# - load_model_dict가 체크포인트에 저장된 "모델 이름/키"에 따라
#   해당 아키텍처를 생성할 수 있도록 build 함수를 제공
#
# [작업자 체크]
# - builders의 key 문자열이 "저장된 체크포인트의 식별자"와 정확히 일치해야 함
#   예) 저장할 때 model_name='patchtst_quantile'로 저장했다면 동일해야 로드됨
#
builders = {
    'patchtst_quantile': build_patchTST_quantile,
    'patchtst': build_patchTST_base,
}


# ------------------------------------------------------------
# (4) 체크포인트 로드
# ------------------------------------------------------------
# load_model_dict(save_root, builders, device=device):
# - save_root 아래에서 체크포인트들을 찾아서 로드
# - 반환값 예: {'patchtst': nn.Module, 'patchtst_quantile': nn.Module, ...}
#
# [작업자 체크]
# - save_root 경로가 "학습에서 저장한 경로"와 동일한지 확인
# - device='cuda'인 경우, 로드 과정에서 map_location이 적절히 처리되는지 확인
#
model_loader = load_model_dict(save_root, builders, device=device)

# base_model: 포인트 예측 모델
base_model = model_loader['patchtst']

# quantile_model: q50 등 분위수 예측 모델
quantile_model = model_loader['patchtst_quantile']


# ------------------------------------------------------------
# (5) 예측 결과 테이블 생성
# ------------------------------------------------------------
# make_forecast_result_table:
# - inference_loader를 순회하며 각 uid(oper_part_no)별로
#   base_forecast(포인트)와 quantile_forecast(q50)를 계산 후 long-format DF 반환
#
# [작업자 체크]
# - make_forecast_result_table 내부에서 "plan_week 포함 여부" 정책 확인 필요
# - max_parts는 대량 결과 생성 시 메모리/시간을 제한하기 위한 장치
#
result_df = make_forecast_result_table(
    inference_loader=inference_loader,
    base_model=base_model,
    quantile_model=quantile_model,
    plan_week=plan_week,
    horizon=horizon,
    device=device,
    max_parts=10_000,
)

# (선택) 결과 확인 예시
# print(result_df.head())
# print(result_df.shape)


In [ ]:
result_df